<a href="https://colab.research.google.com/github/heyggun/bigdata_analysis_test/blob/main/DataManim_type2_regression_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 2유형

## 회귀 Regression

https://www.datamanim.com/dataset/03_dataq/typetwo.html#id18

### 킹카운티 주거지 가격예측문제 데이터

- 데이터 설명 : 중고차 가격 예측 데이터 (종속변수 :G3)
- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv
- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv
- 데이터 출처 :https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?select=vw.csv (참고, 데이터 수정)

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.metrics import r2_score, mean_squared_error

In [2]:
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv')

In [3]:
display(x_train.head(3))
print(x_train.shape)

display(y_train.head(3))
print(y_train.shape)

display(x_test.head(3))
print(x_test.shape)

display(y_test.head(3))
print(y_test.shape)


,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9


(4960, 10)


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990


(4960, 2)


,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,12000,merc,GLS Class,2017,Automatic,12046,Diesel,150.0,37.2,3.0
1,12001,vw,Amarok,2017,Automatic,37683,Diesel,260.0,36.2,3.0
2,12004,merc,GLS Class,2019,Automatic,10000,Diesel,145.0,34.0,3.0


(2672, 10)


,carID,price
0,12000,38000
1,12001,23495
2,12004,59999


(2672, 2)


In [4]:
y_train['price'].describe()

count      4960.000000
mean      23469.943750
std       16406.714563
min         450.000000
25%       12000.000000
50%       18999.000000
75%       30335.750000
max      145000.000000
Name: price, dtype: float64

In [5]:
obj = x_train.select_dtypes(include='object').columns
num = x_train.select_dtypes(include='number').columns


print(len(obj) + len(num))
print(len(x_train.columns))


display(x_train[obj].head(3))
display(x_train[num].head(3))

print(obj)
print(num)

10
10


,brand,model,transmission,fuelType
0,hyundi,Santa Fe,Semi-Auto,Diesel
1,vauxhall,GTC,Manual,Diesel
2,audi,RS4,Automatic,Petrol


,carID,year,mileage,tax,mpg,engineSize
0,13207,2019,4223,145.0,39.8,2.2
1,17314,2015,47870,125.0,60.1,2.0
2,12342,2019,5151,145.0,29.1,2.9


Index(['brand', 'model', 'transmission', 'fuelType'], dtype='object')
Index(['carID', 'year', 'mileage', 'tax', 'mpg', 'engineSize'], dtype='object')


In [6]:
def get_cols(df):

  objs = df.select_dtypes(include='object').columns
  for obj in objs:
    print(df[obj].value_counts().index.unique(), len(df[obj].value_counts().index.unique()))

  return None


get_cols(x_train)
get_cols(x_test)

Index(['merc', 'vw', 'ford', 'bmw', 'hyundi', 'toyota', 'skoda', 'audi',
       'vauxhall'],
      dtype='object') 9
Index([' Arteon', ' Grand C-MAX', ' Scirocco', ' Prius', ' Santa Fe',
       ' CLS Class', ' V Class', ' Scala', ' IX20', ' Galaxy', ' KA', ' Edge',
       ' S Class', ' Combo Life', ' X4', ' Rapid', ' Yeti', ' M4', ' A8',
       ' Kamiq', ' GL Class', ' A7', ' IX35', ' I40', ' Amarok', ' Verso',
       ' 7 Series', ' I800', ' Avensis', ' 6 Series', ' GTC', ' X6', ' Z4',
       ' SLK', ' Caravelle', ' CLA Class', ' Tiguan Allspace', ' Puma', ' CC',
       ' Beetle', ' X-CLASS', ' GLS Class', ' Tourneo Custom', ' GT86',
       ' M Class', ' Zafira Tourer', ' Hilux', ' Grand Tourneo Connect', ' Q8',
       ' Shuttle', ' Mustang', ' Caddy Maxi Life', ' X7', ' Land Cruiser',
       ' i3', ' 8 Series', ' RS6', ' RS3', ' Jetta', ' R8', ' RS4', ' M3',
       ' Tourneo Connect', ' M2', ' RS5', ' Antara', ' Agila', ' M5',
       ' Vivaro', ' SQ5', ' PROACE VERSO', ' i8', ' S4', '

In [7]:
temp = pd.concat([x_train, x_test])

temp = pd.get_dummies(temp)

In [8]:
def do_scaler(df):

  cols = ['year', 'mileage', 'tax', 'mpg', 'engineSize']

  for col in cols:
    Scaler = StandardScaler()
    df[col] = Scaler.fit_transform(df[col].values.reshape(-1,1))

  return df

In [9]:
temp = do_scaler(temp)

display(temp.head(3))


,carID,year,mileage,tax,mpg,engineSize,brand_audi,brand_bmw,brand_ford,brand_hyundi,...,model_ i8,transmission_Automatic,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,13207,0.785152,-0.859188,-0.093598,-0.291383,0.091895,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
1,17314,-0.599466,0.959951,-0.338047,0.279213,-0.163836,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,12342,0.785152,-0.820510,-0.093598,-0.592140,0.986951,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [10]:
x_train_s = temp.iloc[:4960]
x_test_s = temp.iloc[4960:]

In [11]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(x_train_s.drop(columns=['carID']), y_train['price'],
                                                            test_size=0.3,
                                                            random_state=1234)

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

(3472, 113)
(1488, 113)
(3472,)
(1488,)


In [12]:
model = RandomForestRegressor()
ada_model = AdaBoostRegressor()

In [13]:
model.fit(X_train_m, y_train_m)
prediction = model.predict(X_test_m)

print(r2_score(y_test_m, prediction))

0.9579420676990237


In [14]:
ada_model.fit(X_train_m, y_train_m)
ada_prediction = model.predict(X_test_m)

print(r2_score(y_test_m, ada_prediction))

0.9579420676990237


In [15]:
model.fit(x_train_s.drop(columns=['carID']), y_train['price'])
pred = model.predict(x_test_s.drop(columns=['carID']))

print(r2_score(y_test['price'], pred))

0.9600594626708205


In [17]:
pd.DataFrame({'carID': x_test_s['carID'], 'pred':pred, 'real': y_test['price']})

,carID,pred,real
0,12000,38444.72,38000
1,12001,23877.87,23495
2,12004,57256.82,59999
3,12013,14947.75,16713
4,12017,49336.51,46000
...,...,...,...
2667,19618,44833.50,28990
2668,19620,19051.60,17450
2669,19626,20852.58,21995
2670,19630,20422.20,18700
